<a href="https://colab.research.google.com/github/asmaaabdelsattar/instant-training/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers faiss-cpu PyPDF2 gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.6 MB/s eta 0:00:00


In [2]:
import os
import faiss
import pickle
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import gradio as gr


In [3]:
from google.colab import files
uploaded = files.upload()

Saving Cyber Policy- Law and Cyber Crime Investigation_Syllabus_Spring2025.pdf to Cyber Policy- Law and Cyber Crime Investigation_Syllabus_Spring2025.pdf


In [4]:
def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + " "
    return text

def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

pdf_path = "Cyber Policy- Law and Cyber Crime Investigation_Syllabus_Spring2025.pdf"
text = load_pdf(pdf_path)
chunks = chunk_text(text)
print("Number of chunks:", len(chunks))


Number of chunks: 2


In [5]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(chunks, convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

with open("meta.pkl", "wb") as f:
    pickle.dump(chunks, f)

faiss.write_index(index, "faiss.index")
print("Index built and saved!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Index built and saved!


In [6]:
qa_model = pipeline("text2text-generation", model="google/flan-t5-large")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [7]:
def get_answer(question):
    # Load FAISS & chunks
    index = faiss.read_index("faiss.index")
    with open("meta.pkl", "rb") as f:
        docs = pickle.load(f)

    # Encode question & retrieve top 3 relevant chunks
    q_emb = embedding_model.encode([question], convert_to_numpy=True)
    _, idx = index.search(q_emb, 3)
    context = " ".join([docs[i] for i in idx[0]])

    # Smart prompt
    prompt = f"""
You are a helpful and knowledgeable tutor specialized in this course.
Use the context below to answer the question accurately and clearly.

Context:
{context}

Question: {question}
Answer:
"""
    answer = qa_model(prompt, max_length=250, do_sample=False)[0]["generated_text"]
    return answer


In [11]:
# Example question
question = "What are the main objectives of this course?"

# Retrieve relevant context
q_emb = embedding_model.encode([question], convert_to_numpy=True)
_, idx = index.search(q_emb, 3)
context = "\n\n".join([chunks[i] for i in idx[0]])

# Generate answer
prompt = f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
answer = qa_model(prompt, max_length=200, temperature=0.3)[0]['generated_text']

# Print results
print("QUESTION:", question)
print("ANSWER:", answer)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


QUESTION: What are the main objectives of this course?
ANSWER: By the end of this course, students will: Understand the legal frameworks governing cybercrime at national and international levels. 1. Analyze the technical aspects of cybercrime, including malware, phishing, and hacking. 2. Develop skills in digital forensics and evidence collection. 3. Evaluate the ethical and privacy concerns associated with cybercrime investigations. 4. Apply legal and technical knowledge to real-world cybercrime scenarios. 5. Explore the role of law enforcement, private sector, and international organizations in combating cybercrime. 6. Course Schedule: Week 1: Introduction to Cybercrime and the Law Overview of cybercrime: definitions, types, and trends. Historical evolution of cybercrime and its societal impact. Introduction to legal frameworks: national laws and international treaties. Week 2: Legal Foundations of Cybercrime Investigation Key legislation: Computer Fraud and Abuse Act (CFAA), GDPR, C